<a href="https://colab.research.google.com/github/raminbazr/Python/blob/main/Kfold-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kfold for LSTM model**

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from keras.models import Sequential
from keras.layers import Activation, Dense,Flatten,LSTM,Dropout
import tensorflow as tf

# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=18):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
 
# reshape into X=t and Y=t+1
look_back = 18
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Merge inputs and targets
inputs = np.concatenate((trainX, testX), axis=0)
targets = np.concatenate((trainY, testY), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
  # create and fit the LSTM network
  modellstm = Sequential()
  modellstm.add(LSTM(11,return_sequences=True, input_shape=([None, look_back])))
  modellstm.add(Dropout(0.17))
  modellstm.add(LSTM(units=11, return_sequences=False))
  modellstm.add(Dropout(0.17))
  modellstm.add(Dense(1))
  modellstm.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=2e-4, momentum=0.9),metrics=["mae"])
      
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  # Fit data to model
  history = modellstm.fit(inputs[train], targets[train],
                batch_size=4,
                epochs=100,
                verbose=0)

  # Generate generalization metrics
  scores = modellstm.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {modellstm.metrics_names[0]} of {scores[0]}; {modellstm.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1